## Reviews

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import json

### Scrape reviews

In [36]:
# URL = "https://www.imdb.com/title/tt2294629/reviews"  # frozen
URL = "https://www.imdb.com/title/tt2380307/reviews"  # coco


driver = webdriver.Chrome()
wait = WebDriverWait(driver,10)

driver.get(URL)
soup = BeautifulSoup(driver.page_source, 'lxml')

while True:
    try:
        driver.find_element_by_css_selector("button#load-more-trigger").click()
        wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".ipl-load-more__load-indicator")))
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except Exception:break
    
# for elem in soup.find_all(class_='imdb-user-review'):
#     name = elem.find(class_='title').get_text(strip=True)
#     print(name)

driver.quit()

In [29]:
# driver.find_element_by_class_name("ipl-expander").click()

### Clean reviews

In [37]:
names = []
reviews = []
for elem in soup.find_all(class_='imdb-user-review'):
    name = elem.find(class_='title').get_text(strip=True)
    names.append(name)
    try:
        review = elem.find(class_="content").get_text(strip=True)
        reviews.append(review)
    except:
        continue

In [38]:
len(names)

1095

In [39]:
len(reviews)

1095

In [40]:
import json

with open('coco_reviews.json', 'w') as f:
    json.dump(reviews, f)
    
with open('coco_review_titles.json', 'w') as f:
    json.dump(names, f)

## Single Movie TF-IDF

In [7]:
cleaned_reviews = [x.lower() for x in reviews]

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))
X = tfidf.fit_transform(cleaned_reviews)

In [13]:
feature_array = np.array(tfidf.get_feature_names())
tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]

n = 10
top_n = feature_array[tfidf_sorting][:n]

In [14]:
top_n

array(['enough', 'me', 'had', 'smile', 'considering', 'reminded',
       'sadness', 'related', 'tissues', 'grandma'], dtype='<U469')

## Multiple Movies TF-IDF

In [4]:
with open('coco_reviews.json') as f:
    coco = json.load(f)
    
with open('frozen_reviews.json') as f:
    frozen = json.load(f)

In [6]:
coco = " ".join(coco)
frozen = " ".join(frozen)

In [88]:
count = CountVectorizer(ngram_range=(1, 3), stop_words="english").fit([coco, frozen])

In [89]:
t = count.transform([coco, frozen])

In [90]:
t = np.array(t.todense()).T

In [91]:
t.shape

(377711, 2)

In [92]:
w = t.sum(axis=0)
m = 3000
tf = np.divide(t,w)

In [93]:
sum_tij = np.array(t.sum(axis=1)).T

In [94]:
idf = np.log(np.divide(m, sum_tij)).reshape(-1, 1)

In [95]:
tf_idf = np.multiply(tf, idf)

In [96]:
unique_sum = tf_idf.sum(axis=1).reshape(-1, 1)

In [97]:
unique = np.divide(tf_idf, unique_sum)

In [98]:
result = pd.DataFrame(tf_idf, index=count.get_feature_names(), columns=["Coco", "Frozen"])

In [99]:
result.sort_values("Coco", ascending=False).head(20)

,Coco,Frozen
coco,0.004372,0.000002
miguel,0.004176,0.000000
dead,0.003984,0.000096
pixar,0.003784,0.000339
mexican,0.003375,0.000004
family,0.003152,0.000458
culture,0.003107,0.000085
permalink,0.002899,0.000847
helpful sign,0.002899,0.000847
sign vote permalink,0.002899,0.000847


In [100]:
result.sort_values("Frozen", ascending=False).head(20)

,Coco,Frozen
sister,0.000050,0.002330
ice,0.000000,0.002210
anna,0.000000,0.002154
olaf,0.000166,0.002149
snow,0.000014,0.002091
let,0.000373,0.002089
powers,0.000014,0.002083
songs,0.000784,0.002051
hans,0.000007,0.002025
snowman,0.000007,0.002010


### NER

In [55]:
# from transformers import BertTokenizer, BertModel
# from transformers import AutoModelForTokenClassification, AutoTokenizer
# import torch

In [56]:
# model_en = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
# tokenizer_en = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
import json

In [57]:
# with open('data.json', 'w') as f:
#     reviews = json.loads(f)

In [ ]:
label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]

In [ ]:
def predict(tokenizer, model, sequence):
    # Bit of a hack to get the tokens with the special tokens
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="pt")

    outputs = model(inputs)[0]
    predictions = torch.argmax(outputs, dim=2)
    
    return sequence

In [ ]:
reviews[0]

In [ ]:
predict(tokenizer_en, model_en, sequence)

### Sentiment
https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english?text=I+like+you.+I+love+you